In [2]:
import cv2
import numpy as np

Work originally adaptaded from: https://github.com/arunponnusamy/object-detection-opencv

https://www.arunponnusamy.com/yolo-object-detection-opencv-python.html

In [22]:
# download yolo weights

In [1]:
! wget https://pjreddie.com/media/files/yolov3.weights

--2019-08-04 12:33:16--  https://pjreddie.com/media/files/yolov3.weights
wget: /home/antonio/.netrc:4: unknown token "method"
wget: /home/antonio/.netrc:4: unknown token "interactive"
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  2.16MB/s    in 1m 53s  

2019-08-04 12:35:11 (2.09 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [3]:
# Load weights and config file
net = cv2.dnn.readNet('./yolov3.weights', './config.cfg')

In [4]:
net

<dnn_Net 0x7f5a040fbeb0>

In [5]:
# read imput image
image = cv2.imread('./image.jpg')

In [6]:
# set image shape
w, h, _ = image.shape
scale = 0.00392

In [8]:
# prepare image for yolo
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

In [10]:
net.setInput(blob)

In [11]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [12]:
# get the predictions
outs = net.forward(get_output_layers(net))

In [13]:
# read the classes file
classes = None

with open('./classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [14]:
classes[0:10]

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light']

In [15]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

In [16]:
# calculate the round boxes
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * w)
            center_y = int(detection[1] * h)
            w1 = int(detection[2] * w)
            h1 = int(detection[3] * h)
            x = center_x - w1 / 2
            y = center_y - h1 / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w1, h1])

In [17]:
boxes

[[39.5, 98.0, 639, 602]]

In [18]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

In [19]:
# Draw boxes
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [20]:
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

In [21]:
# save image
cv2.imwrite("object-detection.jpg", image)

True